In [1]:
import os
import nltk
from pymorphy2 import MorphAnalyzer
pmm = MorphAnalyzer()
from nltk.tokenize import RegexpTokenizer
from pymorphy2.tokenizers import simple_word_tokenize
tokenizer = RegexpTokenizer(r'\w+')

import RAKE
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('russian')
rake = RAKE.Rake(stop)
from summa import keywords
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.summarization import keywords as kw


def normalize_text(text):
    text = tokenizer.tokenize(text.lower())
    lemmas = [pmm.parse(t)[0].normal_form for t in text]
    return ' '.join(lemmas)


def normalize(text):
    text = tokenizer.tokenize(text.lower())
    return [pmm.parse(t)[0].normal_form for t in text]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qwe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
with open('./texts/meta.txt', 'r', encoding='utf-8') as f:
    meta = [normalize(line) for line in f.read().split('\n')]
meta

[['мотив',
  'цель',
  'смысл',
  'теория',
  'деятельность',
  'самодетерминация',
  'интерес',
  'внутренний',
  'внешний',
  'мотивация',
  'качество'],
 ['компьютерный',
  'игра',
  'игровой',
  'компьютерный',
  'зависимость',
  'младший',
  'подростковый',
  'возраст',
  'опросник',
  'подросток',
  'компьютер'],
 ['копинга',
  'трудный',
  'жизненный',
  'ситуация',
  'защитный',
  'механизм',
  'стратегия',
  'совладаний',
  'когнитивный',
  'оценивание',
  'ситуация'],
 ['моральный',
  'развитие',
  'моральный',
  'ориентация',
  'принцип',
  'справедливость',
  'принцип',
  'забота',
  'подростковый',
  'возраст',
  'юношеский'],
 ['восприятие',
  'развитие',
  'социализация',
  'ребёнок',
  'мультипликационный',
  'фильм',
  'образ',
  'герой',
  'категория',
  'оценка',
  'персонаж']]

In [3]:
data = []
for file in os.listdir('./texts'):
    if file.startswith('text'):
        path = os.path.join('./texts/', file)
        with open(path, 'r', encoding='utf-8') as f:
            nb = int(file[4])
            data.append(tuple([f.read(), meta[nb-1]]))

corpus = [d[0] for d in data]       

In [4]:
data = []
for file in os.listdir('./texts'):
    if file.startswith('text'):
        path = os.path.join('./texts/', file)
        with open(path, 'r', encoding='utf-8') as f:
            data.append(tuple([f.read(), meta[nb-1]]))

In [5]:
def textRank(text):
    text = normalize_text(text)
    result = []
    kw(text, pos_filter=[], scores=True)
    keywords.keywords(normalize_text(text), additional_stopwords=stop, scores=True)
    G = keywords.get_graph(text)
    result = [(edge, G.edge_weight(edge)) for edge in G.edges() if all(map(lambda x: x not in stop, edge))]
    result = sorted(result, key = lambda x: x[1], reverse=True)[:11]
    for i in result:
        print(i)
    return result

In [6]:
 def TfIdf(text, ng = (1,3)):
    vectorizer = TfidfVectorizer(ngram_range = ng)
    X = vectorizer.fit_transform([normalize_text(text)])
    names = vectorizer.get_feature_names()
    words = {}
    for col in X.nonzero()[1]:
        if all(map(lambda i: i not in stop, names[col].split(' '))):
            words[tuple(names[col].split(' '))] =  float(X[0, col])
    top = sorted(words.items(), key = lambda x: x[1], reverse=True)
    for i in top[:11]:
        print(i[0], i[1])
    return [i[0] for i in top[:7]]

In [7]:
def Rake(text):
    pred = []
    for i in rake.run(normalize_text(text), minFrequency=2, maxWords=3)[:11]:
            print(i[0], i[1])
            pred.append(i[0])
    return pred

In [8]:
textrank_res = []
for d in corpus:
    textrank_res.append(textRank(d))
    print('============')

(('леонтьев', 'далее'), 1)
(('далее', 'леонтьев'), 1)
(('мотив', 'понятие'), 1)
(('понятие', 'мотив'), 1)
(('леонтьев', 'наш'), 1)
(('наш', 'леонтьев'), 1)
(('качество', 'проблема'), 1)
(('проблема', 'качество'), 1)
(('мотивация', 'качество'), 1)
(('качество', 'мотивация'), 1)
(('вестник', 'мотивация'), 1)
(('опросник', 'тест'), 1)
(('тест', 'опросник'), 1)
(('степень', 'опросник'), 1)
(('опросник', 'степень'), 1)
(('увлечённость', 'степень'), 1)
(('степень', 'увлечённость'), 1)
(('младший', 'увлечённость'), 1)
(('увлечённость', 'младший'), 1)
(('подросток', 'младший'), 1)
(('младший', 'подросток'), 1)
(('компьютерный', 'подросток'), 1)
(('битюцкий', 'е'), 1)
(('е', 'битюцкий'), 1)
(('подход', 'современный'), 1)
(('современный', 'подход'), 1)
(('совладаний', 'изучение'), 1)
(('изучение', 'совладаний'), 1)
(('жизненный', 'трудный'), 1)
(('трудный', 'жизненный'), 1)
(('ситуация', 'жизненный'), 1)
(('жизненный', 'ситуация'), 1)
(('вестник', 'ситуация'), 1)
(('справедливость', 'мораль'), 1

In [9]:
tfidf_res = []

for d in corpus:
    tfidf_res.append(TfIdf(d, ng = (3,3)))
    print('==================')

('внутренний', 'внешний', 'мотивация') 0.06523280730534423
('деятельность', 'теория', 'самодетерминация') 0.06523280730534423
('теория', 'деятельность', 'теория') 0.06523280730534423
('работа', 'наш', 'леонтьев') 0.04892460547900816
('леонтьев', 'наш', '2009') 0.04892460547900816
('качественный', 'различие', 'мотивация') 0.04892460547900816
('деятельность', 'леонтьев', 'наш') 0.04892460547900816
('сантиметр', 'леонтьев', 'далее') 0.04892460547900816
('потребность', 'личность', 'целое') 0.04892460547900816
('глубинный', 'потребность', 'личность') 0.04892460547900816
('мотив', 'цель', 'смысл') 0.04892460547900816
('пункт', 'номер', 'вопрос') 0.09508728264112049
('ки', 'суммарный', 'показатель') 0.0760698261128964
('ки', 'высокий', 'показатель') 0.0760698261128964
('степень', 'увлечённость', 'ки') 0.0760698261128964
('бабаев', 'войскунский', '1998') 0.0760698261128964
('эмоциональный', 'отношение', 'ки') 0.0760698261128964
('связь', 'шкала', 'опросник') 0.057052369584672295
('иса', 'иц', 

In [10]:
for d in corpus:
    TfIdf(d, ng = (1,3))
    print('==================')

('мотив',) 0.3285343612824541
('деятельность',) 0.283734221107574
('цель',) 0.19040059574324045
('мотивация',) 0.18293390571409376
('это',) 0.17173387067037374
('смысл',) 0.1680005256558004
('леонтьев',) 0.1493338005829337
('действие',) 0.13440042052464032
('который',) 0.13440042052464032
('теория',) 0.13440042052464032
('наш',) 0.13066707551006698
('ки',) 0.3338962569615573
('показатель',) 0.19573228856367153
('шкала',) 0.18997545654709294
('опросник',) 0.18997545654709294
('игра',) 0.16119129646420008
('компьютерный',) 0.15543446444762152
('результат',) 0.14967763243104293
('уровень',) 0.14392080041446437
('подросток',) 0.13240713638130722
('компьютерный', 'игра') 0.10362297629841434
('младший',) 0.10362297629841434
('ситуация',) 0.45480582865146224
('копинга',) 0.33990751404477704
('стратегия',) 0.16756004213474926
('совладаний',) 0.16756004213474926
('исследование',) 0.1053234550561281
('трудный',) 0.09574859550557101
('поведение',) 0.09096116573029245
('который',) 0.08617373595501

In [11]:
tfidf_res = []
for d in corpus:
    tfidf_res.append(TfIdf(d, ng = (2,2)))
    print('==================')

('наш', 'леонтьев') 0.27425111706914745
('теория', 'самодетерминация') 0.16205747826813258
('леонтьев', 'наш') 0.1246595986677943
('теория', 'деятельность') 0.1246595986677943
('мотив', 'цель') 0.09972767893423544
('понятие', 'мотив') 0.09972767893423544
('леонтьев', 'далее') 0.09972767893423544
('деятельность', 'который') 0.08726171906745601
('смысл', 'действие') 0.08726171906745601
('внешний', 'мотив') 0.08726171906745601
('внешний', 'мотивация') 0.08726171906745601
('компьютерный', 'игра') 0.26649855873198514
('младший', 'подросток') 0.207276656791544
('шкала', 'опросник') 0.17766570582132343
('степень', 'увлечённость') 0.11844380388088228
('высокий', 'показатель') 0.103638328395772
('отношение', 'ки') 0.103638328395772
('номер', 'вопрос') 0.08883285291066172
('получить', 'результат') 0.08883285291066172
('увлечённость', 'ки') 0.08883285291066172
('компьютерный', 'зависимость') 0.08883285291066172
('пункт', 'номер') 0.07402737742555143
('жизненный', 'ситуация') 0.17946464785582605
(

In [12]:
rake_res = []
for d in data:
    rake_res.append(Rake(d[0]))
    print('==================')

данный момент 4.933333333333334
теория самодетерминация 4.885714285714286
понятие мотив 4.544444444444444
субъект делать 4.504761904761905
смысловой связь 4.5
мотив возникать 4.377777777777778
теория деятельность 4.322751322751323
функция мотив 4.294444444444444
внешний мотивация 4.272727272727273
самый процесс 4.2
начать делать 4.183333333333334
основание получить дать 8.333333333333334
ио отрицательно коррелировать 8.333333333333334
ки целевой направленность 8.117647058823529
ки родительский отношение 8.117647058823529
компьютерный игра 4.96969696969697
база методика 4.333333333333334
результат исследование 4.333333333333334
ки самоконтроль 4.117647058823529
достижение весь 4.0
сделать вывод 4.0
игра 2.3333333333333335
исследование совладаний 4.96078431372549
изменение ситуация 4.1923076923076925
внутренний ресурс 4.0
совладаний 2.2941176470588234
работа 2.2
ситуация 2.1923076923076925
результат 2.0
помощь 1.75
определение 1.6666666666666667
образ 1.5
влиять 1.3333333333333333
психол

In [13]:
print(tfidf_res, '\n', textrank_res, '\n', rake_res)

[[('наш', 'леонтьев'), ('теория', 'самодетерминация'), ('леонтьев', 'наш'), ('теория', 'деятельность'), ('мотив', 'цель'), ('понятие', 'мотив'), ('леонтьев', 'далее')], [('компьютерный', 'игра'), ('младший', 'подросток'), ('шкала', 'опросник'), ('степень', 'увлечённость'), ('высокий', 'показатель'), ('отношение', 'ки'), ('номер', 'вопрос')], [('жизненный', 'ситуация'), ('копинга', 'стратегия'), ('трудный', 'жизненный'), ('et', 'al'), ('стрессовый', 'ситуация'), ('совладаний', 'трудный'), ('копинга', 'стиль')], [('моральный', 'развитие'), ('моральный', 'ориентация'), ('моральный', 'выбор'), ('развитие', 'моральный'), ('моральный', 'мышление'), ('моральный', 'сознание'), ('моральный', 'норма')], [('младший', 'школьник'), ('герой', 'мультфильм'), ('зарубежный', 'мультфильм'), ('образ', 'герой'), ('отечественный', 'зарубежный'), ('школьный', 'возраст'), ('младший', 'школьный')]] 
 [[(('леонтьев', 'далее'), 1), (('далее', 'леонтьев'), 1), (('мотив', 'понятие'), 1), (('понятие', 'мотив'), 1)

In [14]:
from sklearn.metrics import f1_score
metrics_true = []
y_pred = []
for line in meta:
    metrics_true.append([1] * len(line))
    y_pred.append([0] * len(line))
(metrics_true, y_pred)

([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [15]:
from copy import deepcopy

In [16]:
rake_pred = deepcopy(y_pred)
tfidf_pred = deepcopy(y_pred)
textrank_pred = deepcopy(y_pred)

### TfIdf

In [17]:
for nb,text in enumerate(tfidf_res):
    print(nb, text, meta[nb])
    for idx, words in enumerate(text):
        tfidf_pred[nb][idx] = int(True in (map(lambda x: x in meta[nb], words)))
tfidf_pred            

0 [('наш', 'леонтьев'), ('теория', 'самодетерминация'), ('леонтьев', 'наш'), ('теория', 'деятельность'), ('мотив', 'цель'), ('понятие', 'мотив'), ('леонтьев', 'далее')] ['мотив', 'цель', 'смысл', 'теория', 'деятельность', 'самодетерминация', 'интерес', 'внутренний', 'внешний', 'мотивация', 'качество']
1 [('компьютерный', 'игра'), ('младший', 'подросток'), ('шкала', 'опросник'), ('степень', 'увлечённость'), ('высокий', 'показатель'), ('отношение', 'ки'), ('номер', 'вопрос')] ['компьютерный', 'игра', 'игровой', 'компьютерный', 'зависимость', 'младший', 'подростковый', 'возраст', 'опросник', 'подросток', 'компьютер']
2 [('жизненный', 'ситуация'), ('копинга', 'стратегия'), ('трудный', 'жизненный'), ('et', 'al'), ('стрессовый', 'ситуация'), ('совладаний', 'трудный'), ('копинга', 'стиль')] ['копинга', 'трудный', 'жизненный', 'ситуация', 'защитный', 'механизм', 'стратегия', 'совладаний', 'когнитивный', 'оценивание', 'ситуация']
3 [('моральный', 'развитие'), ('моральный', 'ориентация'), ('мора

[[0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
 [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]]

In [18]:
y_true = [item for inner in metrics_true for item in inner]
y_pred = [item for inner in tfidf_pred for item in inner]
# sklearn.metrics.accuracy_score(y_true, y_pred), sklearn.metrics.recall_score(y_true, y_pred), f1_score(y_true, y_pred) 
m = sklearn.metrics.precision_recall_fscore_support(y_true, y_pred)
print('\t'.join([str(item) for item in m]))

[0. 1.]	[0.  0.4]	[0.         0.57142857]	[ 0 55]


c:\users\qwe\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


#### Ошибки

* Не выкидывает местоимения (какой-то), нужно их включать в стоп-слова
* Самые частотные != ключевые
* Включает в ключевые слова фамилии ученых

### Rake

In [19]:
for nb,text in enumerate(rake_res):
    print(nb, text, meta[nb])
    for idx, words in enumerate(text):
        line = words.split()
        rake_pred[nb][idx] = int(True in (map(lambda x: x in meta[nb], line)))
rake_pred  

0 ['данный момент', 'теория самодетерминация', 'понятие мотив', 'субъект делать', 'смысловой связь', 'мотив возникать', 'теория деятельность', 'функция мотив', 'внешний мотивация', 'самый процесс', 'начать делать'] ['мотив', 'цель', 'смысл', 'теория', 'деятельность', 'самодетерминация', 'интерес', 'внутренний', 'внешний', 'мотивация', 'качество']
1 ['основание получить дать', 'ио отрицательно коррелировать', 'ки целевой направленность', 'ки родительский отношение', 'компьютерный игра', 'база методика', 'результат исследование', 'ки самоконтроль', 'достижение весь', 'сделать вывод', 'игра'] ['компьютерный', 'игра', 'игровой', 'компьютерный', 'зависимость', 'младший', 'подростковый', 'возраст', 'опросник', 'подросток', 'компьютер']
2 ['исследование совладаний', 'изменение ситуация', 'внутренний ресурс', 'совладаний', 'работа', 'ситуация', 'результат', 'помощь', 'определение', 'образ', 'влиять'] ['копинга', 'трудный', 'жизненный', 'ситуация', 'защитный', 'механизм', 'стратегия', 'совладан

[[0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
 [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0]]

In [20]:
#RAKE
y_pred = [item for inner in rake_pred for item in inner]

m = sklearn.metrics.precision_recall_fscore_support(y_true, y_pred)
print('\t'.join([str(item) for item in m]))

[0. 1.]	[0.         0.38181818]	[0.         0.55263158]	[ 0 55]


##### Ошибки

* Лишние слова, которые берет во внимание - "самый", "е"
* В биграммах и просто словах - повторение одних и тех же слов ("изменение ситуации" и "ситуация", "младший школьный возраст" и "младший школьник")
* тоже берет фамилии ("мочалов")
* Самыйй частотные слова != ключевые

### TextRank

In [21]:
for nb,text in enumerate(textrank_res):
    print(nb, text, meta[nb])
    for idx, words in enumerate(text):
        textrank_pred[nb][idx] = int(True in (map(lambda x: x in meta[nb], words[0])))
textrank_pred  

0 [(('леонтьев', 'далее'), 1), (('далее', 'леонтьев'), 1), (('мотив', 'понятие'), 1), (('понятие', 'мотив'), 1), (('леонтьев', 'наш'), 1), (('наш', 'леонтьев'), 1), (('качество', 'проблема'), 1), (('проблема', 'качество'), 1), (('мотивация', 'качество'), 1), (('качество', 'мотивация'), 1), (('вестник', 'мотивация'), 1)] ['мотив', 'цель', 'смысл', 'теория', 'деятельность', 'самодетерминация', 'интерес', 'внутренний', 'внешний', 'мотивация', 'качество']
1 [(('опросник', 'тест'), 1), (('тест', 'опросник'), 1), (('степень', 'опросник'), 1), (('опросник', 'степень'), 1), (('увлечённость', 'степень'), 1), (('степень', 'увлечённость'), 1), (('младший', 'увлечённость'), 1), (('увлечённость', 'младший'), 1), (('подросток', 'младший'), 1), (('младший', 'подросток'), 1), (('компьютерный', 'подросток'), 1)] ['компьютерный', 'игра', 'игровой', 'компьютерный', 'зависимость', 'младший', 'подростковый', 'возраст', 'опросник', 'подросток', 'компьютер']
2 [(('битюцкий', 'е'), 1), (('е', 'битюцкий'), 1),

[[0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1],
 [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]]

In [22]:
#text_rank
y_pred = [item for inner in textrank_pred for item in inner]
# sklearn.metrics.accuracy_score(y_true, y_pred), sklearn.metrics.recall_score(y_true, y_pred), f1_score(y_true, y_pred) \
m = sklearn.metrics.precision_recall_fscore_support(y_true, y_pred)
print('\t'.join([str(item) for item in m]))

[0. 1.]	[0.         0.69090909]	[0.        0.8172043]	[ 0 55]


#### Ошибки:

* Считает биграммы подряд, в том порядке, в котором они в тексте
* В данном случае для всех биграмм частота равна 1, т.е. они встретилсь в тексте один раз, что сомнительно
* из-за того, что биграммы повторяются, метрики становятся высокими, хотя набор ключевый слов получается меньшим

### Итог

* Для поиска ключевых слов недостаточно знать частотность
* В исходных текстах встречалось много примеров описывамых явлений, поэтому частостность ключевых слов уменьшалась засчет преобладания других слов
* Системы не всегда считают частоты биграмм корректно, включая повторы слов или лишние слова